In [1]:
from bse_dsdm.preprocessing.feature_scaling import *
from bse_dsdm.preprocessing.outlier_detection import *
from bse_dsdm.preprocessing.value_encoder import *
from bse_dsdm.preprocessing.value_imputer import *
from bse_dsdm.preprocessing.data_loader import *
#from bse_dsdm.preprocessing.preprocessing_pipeline import *
from bse_dsdm.accuracy_testing.error_testing_methods import *
from bse_dsdm.preprocessing.exploratory_analysis import *
from bse_dsdm.accuracy_testing.cross_validation import *
from bse_dsdm.model_development.model import *
import pandas as pd
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV

LOAD DATA

In [2]:
#Load the data
data=MLDataLoader()
data.load_data("../train.csv")


PREPROCESSING

In [3]:
#Delete columns that exceed 80% of null values
data.data=remove_columns_with_na(data.data,80)


In [4]:
#Returns the categorical and numerical columns with null values
categorical_columns_na=get_columns_by_type(data.data)[0]
print(categorical_columns_na)
numerical_columns_na=get_columns_by_type(data.data)[1]
print(numerical_columns_na)


['club_joined', 'player_traits']
['value_eur', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']


In [5]:
#Impute numerical values with the mean
data.data[numerical_columns_na]=impute_values(data.data[numerical_columns_na], impute_type='mean')

In [6]:
#impute categorical values with the mode

data.data=impute_categorical(data.data,categorical_columns_na)

          id   short_name  overall  potential   value_eur  wage_eur  \
0     216302    E. García       71         71   1400000.0     10000   
1     237867   D. Cancola       65         71   1000000.0      2000   
2     253472      E. Kahl       65         77   1600000.0      2000   
3     223994    S. Mugoša       72         72   2300000.0      5000   
4     251635  A. Țigănașu       65         65    525000.0      3000   
...      ...          ...      ...        ...         ...       ...   
6890  242007    C. Whelan       59         66    425000.0      2000   
6891  187961     Paulinho       83         83  28500000.0     61000   
6892  257234   Y. Hamache       70         80   3400000.0      6000   
6893  232511    S. Sasaki       71         71   1300000.0      7000   
6894  246750    S. Haudum       66         70   1100000.0      4000   

     birthday_date  height_cm  weight_kg                  club_name  ...  \
0       1989-12-28        176         73  Club Atlético de San Luis  ..

Features

In [7]:
#Encode ordinal columns
mapping={'work_rate': {'Low/Low': 1, 'Low/Medium': 2, 'Low/High': 3, 'Medium/Low': 4, 'Medium/Medium': 5, 'Medium/High': 6, 'High/Low': 7, 'High/Medium': 8, 'High/High': 9}}

data.data = encode_ordinal_columns(data.data, columns=['work_rate'], mapping=mapping)

In [8]:
# Target encode column  
#Each row in this column will contain the mean height cm of individuals belonging to a particular nationality
data.data['nationality_name'] = target_encode_column(data.data, 'nationality_name', 'height_cm', compute_type='mean')

In [9]:
# One-hot enconding
#Replaces categorical columns with binary columns(0s and 1s)
data.data=encode_categorical_columns(data.data,column='preferred_foot')

c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [10]:
#The year is extracted for the birthday date and substracted for the year of the data
data.data = convert_to_age(data.data, 'birthday_date',2021)

In [11]:
#One hot encoded columns derived from the processed strings in the specified column
data.data=extract_string(data.data,'body_type')

In [12]:
#we split the data between train and test
data.split_data()


In [13]:

feature_columns=['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle']
target_column=['position']
X=data.data[feature_columns]
y=data.data['position']


In [14]:
data.data=scale_features(data.data,feature_columns,'min_max')

In [15]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)
model.train()
predictions = model.predict()
print(f"predictions:{predictions}")

accuracy = model.get_accuracy(predictions, data.test_data[target_column])
print(f"Accuracy: {accuracy}")


c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


predictions:['LCM' 'RCB' 'LCB' ... 'RCM' 'ST' 'GK']
Accuracy: 0.3836113125453227


In [17]:

model = RandomForestModel(feature_columns, target_column, data.train_data, data.test_data)

param_grid = {
    'n_estimators': [20, 100, 200],
    'max_depth': [None, 10, 15],
    'min_samples_split': [1, 3, 5],
    'min_samples_leaf': [1, 2, 4],
}


best_model = perform_grid_search(model, param_grid)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
225 fits failed out of a total of 405.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\USUARIO\anaconda3\Lib\s

Best Hyperparameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 15}
Best Accuracy: 0.3738251737685101
Test Accuracy with Best Hyperparameters: 0.3828861493836113


In [21]:
evaluate_classification_metrics(model.test_data[model.target_column], best_model.predict(model.test_data[feature_columns]))

------ Multiclass Classification Metrics Summary ------
Accuracy: 0.3829 - The proportion of correctly classified instances.
Precision: 0.3157 - Weighted average precision across all classes.
Recall: 0.3829 - Weighted average recall across all classes.
F1 Score: 0.3354 - Weighted average F1 score across all classes.


c:\Users\USUARIO\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
